I have updated the document and it gets wayyyyyyyyyyyyyy more exciting than the first version. Anyway, here are links to each 'chapter'
- [Test Run 1 - Reward for Surviving](#Test-Agent---Run-1)
- [Test Run 2 - Reward for Entering a new state](#Test-Agent---Run-2)
- [Test Run 3 - Punish for Dying](#Test-Agent---Run-3)
- [Test Run 4 - Punish for Dying (Different Level)](#Test-Agent---Run-4)
- [Test Run 5 - Punish for Dying & Reward for Staying Alive](#Test-Agent---Run-5)
- [Test Run 6 - Punish for Dying, Reward for Staying Alive & More Advanced Decision Making](#Test-Agent---Run-6-(Solid-Agent))
- [Test Run 7 - Punish for Dying, Reward for Staying Alive, Reward for Entering a new state & More Advanced Decision Making](#Test-Agent---Run-7-(Pretty-Solid))
- [Test Run 8 - Look a step into the future](#Test-Agent---Run-8-(Amazing))
- [Test Run 9 - Look 3 steps into the future](#Test-Agent---Run-9-(Amazing))
- [Test Run 10 - Look 5 steps into the future](#Test-Agent---Run-10-(Amazing))
- [Test Run 11 - Look x steps into the future](#Test-Agent---Run-11-(Perfect))
- [Test Run 12 - Nailing 8x8 stage](#Test-Agent---Run-12-(THE-ULTIMATE-TEST))

#### Import dependencies

In [1]:
import gym # 0.26.2
import pandas as pd # 2.0.3
import numpy as np # 1.23.4
import random as rnd # N/A
import time # N/A

print(gym.__version__)
print(pd.__version__)
print(np.__version__)

c:\Users\kaan-\anaconda3\envs\gymenv38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\kaan-\anaconda3\envs\gymenv38\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\kaan-\anaconda3\envs\gymenv38\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


0.26.2
2.0.3
1.23.4


#### Create Environment

I made `is_slippery = False` because it increases the amount of luck and decreases the amount of skill required to complete this challenge.

In [5]:
env = gym.make('FrozenLake-v1', 
               render_mode='human',
               desc=["SFHF", "HFFF", "HFHF", "FFHG"], 
               is_slippery = False)
env.reset()
env.render()

#### Check action and obervation space

In [3]:
print(env.observation_space)
print(env.action_space)

Discrete(16)
Discrete(4)


#### Random Agent

In [4]:
env.reset()
while True:
    env.render()
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    print(f'location: {observation}')
    print(f'reward: {reward}')
    print(f'dead: {terminated}')
    print(f'crashed {truncated}')
    print(f'chance: {info["prob"]}')
    print()
    if terminated or truncated:
        break

location: 0
reward: 0.0
dead: False
crashed False
chance: 1.0

location: 0
reward: 0.0
dead: False
crashed False
chance: 1.0

location: 0
reward: 0.0
dead: False
crashed False
chance: 1.0

location: 0
reward: 0.0
dead: False
crashed False
chance: 1.0

location: 0
reward: 0.0
dead: False
crashed False
chance: 1.0

location: 0
reward: 0.0
dead: False
crashed False
chance: 1.0

location: 0
reward: 0.0
dead: False
crashed False
chance: 1.0

location: 4
reward: 0.0
dead: True
crashed False
chance: 1.0



#### Create Q Table
In the table we keep track of the action and reward per state

In [8]:
def create_q_table(len=16):
    q_table = []
    for _ in range(len):
        base_action = None
        base_reward = 0
        q_table.append([base_action, base_reward])
    return q_table

In [6]:
states = env.observation_space.n
q_table = create_q_table(states)
print(q_table)

[[None, 0], [None, 0], [None, 0], [None, 0], [None, 0], [None, 0], [None, 0], [None, 0], [None, 0], [None, 0], [None, 0], [None, 0], [None, 0], [None, 0], [None, 0], [None, 0]]


#### Create Agent

In [6]:
import random as rnd

class Agent():
    def __init__(self, q_table, exploration_rate = 0):
        self.q_table = q_table
        self.exploration_rate = exploration_rate*100

    def step(self, state):
        action, _ = self.q_table[state]
        if action == None:
            return env.action_space.sample()
        else:
            rng = rnd.randint(1, 100)
            if rng <= self.exploration_rate:
                return env.action_space.sample()
            else:
                return action

    def learn(self, state, new_action, new_reward):
        _, old_reward = self.q_table[state]
        if new_reward > old_reward:
            self.q_table[state] = [new_action, new_reward]    

    def evaluate(self):
        df = pd.DataFrame(self.q_table, columns=['Action', 'Reward'])
        print(df)


## Test Agent - Run 1
Parameters
- `env.desc=["SFHF", "HFFF", "HFHF", "FFHG"]`
- `is_slippery = False`
- `exploration_rate = 0.0`
- Starting point = 0
- Goal = 15
- +1 reward for surviving after an action

Our expected outcome is to stay alive for 30 turns because staying alive is rewarded. The agent can do this by moving up or left repeatedly. It could also find a movement loop that will make it survive.

In [187]:
agent1 = Agent(create_q_table(), exploration_rate=0.0)
agent1.q_table

[[None, 0],
 [None, 0],
 [None, 0],
 [None, 0],
 [None, 0],
 [None, 0],
 [None, 0],
 [None, 0],
 [None, 0],
 [None, 0],
 [None, 0],
 [None, 0],
 [None, 0],
 [None, 0],
 [None, 0],
 [None, 0]]

In [189]:

observation = 0  # We consider that we always start top left
max_moves = 30

for move in range(max_moves):
    # Render the environment
    env.render()

    # Get an action based of the Q-table
    action = agent1.step(observation)
    old_observation = observation

    # Execute action
    observation, reward, terminated, _, _ = env.step(action)

    # Give reward if still alive
    if terminated==False:
        reward=1
    else:
        reward=-1

    # Make the agent learn
    agent1.learn(old_observation, action, reward)
    
    # Print usefull info to track progress
    print(f'turn: {move+1}')
    print(f'old_location: {old_observation}')
    print(f'action: {action}')
    print(f'new_location: {observation}')
    print(f'reward: {reward}')
    print(f'dead: {terminated}')
    print()
    
    # stop if dead
    if terminated or truncated:
        break


turn: 1
old_location: 0
action: 3
new_location: 0
reward: 1
dead: False

turn: 2
old_location: 0
action: 3
new_location: 0
reward: 1
dead: False

turn: 3
old_location: 0
action: 3
new_location: 0
reward: 1
dead: False

turn: 4
old_location: 0
action: 3
new_location: 0
reward: 1
dead: False

turn: 5
old_location: 0
action: 3
new_location: 0
reward: 1
dead: False

turn: 6
old_location: 0
action: 3
new_location: 0
reward: 1
dead: False

turn: 7
old_location: 0
action: 3
new_location: 0
reward: 1
dead: False

turn: 8
old_location: 0
action: 3
new_location: 0
reward: 1
dead: False

turn: 9
old_location: 0
action: 3
new_location: 0
reward: 1
dead: False

turn: 10
old_location: 0
action: 3
new_location: 0
reward: 1
dead: False

turn: 11
old_location: 0
action: 3
new_location: 0
reward: 1
dead: False

turn: 12
old_location: 0
action: 3
new_location: 0
reward: 1
dead: False

turn: 13
old_location: 0
action: 3
new_location: 0
reward: 1
dead: False

turn: 14
old_location: 0
action: 3
new_location

## Evaluation
You might have seen it from the output, the Agent taught itself to always try to move up since moving up makes it stay alive. We can confirm this by looking in our Q-table. Even when it moved to the right it learned itself to move left to get back on its "Safe Spot"

In [191]:
# 0: LEFT
# 1: DOWN
# 2: RIGHT
# 3: UP
agent1.evaluate()


    Action  Reward
0      3.0       1
1      NaN       0
2      NaN       0
3      NaN       0
4      NaN       0
5      NaN       0
6      NaN       0
7      NaN       0
8      NaN       0
9      NaN       0
10     NaN       0
11     NaN       0
12     NaN       0
13     NaN       0
14     NaN       0
15     NaN       0


## Test Agent - Run 2
Parameters
- `env.desc = ["SFHF", "HFFF", "HFHF", "FFHG"]`
- `is_slippery = False`
- `exploration_rate = 0.2`
- Starting point = 0
- Goal = 15
- +0 reward for surviving after an action
- +1 reward for entering a new state

I expect the agent to survive by moving on at least 2 tiles. 

In [9]:
states = env.observation_space.n
agent2 = Agent(create_q_table(states), exploration_rate=0.2)

In [11]:
observation = 0  # We consider that we always start top left
max_moves = 20
episodes = 10
agent = agent2

for episode in range(episodes):
    env.reset()
    for move in range(max_moves):
        # Render the environment
        env.render()

        # Get an action based of the Q-table
        action = agent.step(observation)
        old_observation = observation

        # Execute action
        observation, reward, terminated, _, _ = env.step(action)

        # Give reward
        if terminated == False:
            if observation == old_observation:
                reward = 0
            else:
                reward = 1
        else:
            reward = -1

        # Make the agent learn
        agent.learn(old_observation, action, reward)

        # Print usefull info to track progress
        print(f'turn: {move+1}')
        print(f'old_location: {old_observation}')
        print(f'action: {action}')
        print(f'new_location: {observation}')
        print(f'reward: {reward}')
        print(f'dead: {terminated}')
        print()

        # stop if dead
        if terminated:
            break


turn: 1
old_location: 0
action: 3
new_location: 0
reward: 0
dead: False

turn: 2
old_location: 0
action: 2
new_location: 1
reward: 1
dead: False

turn: 3
old_location: 1
action: 0
new_location: 0
reward: 1
dead: False

turn: 4
old_location: 0
action: 2
new_location: 1
reward: 1
dead: False

turn: 5
old_location: 1
action: 0
new_location: 0
reward: 1
dead: False

turn: 6
old_location: 0
action: 2
new_location: 1
reward: 1
dead: False

turn: 7
old_location: 1
action: 0
new_location: 0
reward: 1
dead: False

turn: 8
old_location: 0
action: 2
new_location: 1
reward: 1
dead: False

turn: 9
old_location: 1
action: 0
new_location: 0
reward: 1
dead: False

turn: 10
old_location: 0
action: 2
new_location: 1
reward: 1
dead: False

turn: 11
old_location: 1
action: 0
new_location: 0
reward: 1
dead: False

turn: 12
old_location: 0
action: 2
new_location: 1
reward: 1
dead: False

turn: 13
old_location: 1
action: 0
new_location: 0
reward: 1
dead: False

turn: 14
old_location: 0
action: 2
new_location

## Evaluation
As we can see the agent came suprisingly close to the finish, but still it was moving back and forth a lot. to fix this, the agent has to come up with a different policy and we have to provide a different reward system.

In [12]:
agent2.evaluate()

    Action  Reward
0      2.0       1
1      0.0       1
2      3.0       1
3      NaN       0
4      2.0       1
5      3.0       1
6      NaN       0
7      NaN       0
8      NaN       0
9      NaN       0
10     NaN       0
11     NaN       0
12     NaN       0
13     NaN       0
14     NaN       0
15     NaN       0


## Test Agent - Run 3
Let's throw things around, I will save the worst possible actions in my q_table and prevent these actions. so instead of telling our agent where to move, we tell it where not to move.

Parameters
- `env.desc = ["SFHF", "HFFF", "HFHF", "FFHG"]`
- `is_slippery = False`
- `exploration_rate = 0.0`
- Starting point = 0
- Goal = 15
- -1 reward for dying after an action

I expect the agent to get to the goal at some point. it might fail but as it plays it should learn where the holes are.

In [80]:
class Agent_New():
    def __init__(self, q_table, exploration_rate=0):
        self.q_table = q_table
        self.exploration_rate = exploration_rate*100

    def step(self, state):
        action, _ = self.q_table[state]
        if action == None or action is None:
            action = env.action_space.sample()
            return action
        else:
            rng = rnd.randint(1, 100)
            if rng <= self.exploration_rate:
                action = env.action_space.sample()
                return action
            else:
                different_action = env.action_space.sample()
                while different_action == action: 
                    different_action = env.action_space.sample()
                return different_action
                # print(f'dif {different_action}')
                # print(f'accc {action}')

    def learn(self, state, new_action, new_reward):
        _, old_reward = self.q_table[state]
        if new_reward < old_reward:
            self.q_table[state] = [new_action, new_reward]

    def evaluate(self):
        df = pd.DataFrame(self.q_table, columns=['Action', 'Reward'])
        print(df)


In [81]:
states = env.observation_space.n
agent3 = Agent_New(create_q_table(states), exploration_rate=0.0)

In [85]:
observation = 0  # We consider that we always start top left
max_moves = 100
episodes = 10
agent = agent3

for episode in range(episodes):
    env.reset()
    for move in range(max_moves):
        # Get an action based of the Q-table
        action = agent.step(observation)
        old_observation = observation

        # Execute action
        observation, reward, terminated, _, _ = env.step(action)

        # Print usefull info to track progress
        print(f'turn: {move}')
        print(f'old_location: {old_observation}')
        print(f'action: {action}')
        print(f'new_location: {observation}')
        print(f'reward: {reward}')
        print(f'dead: {terminated}')
        print()

        # stop if dead
        if terminated:
            reward = -1
            agent.learn(old_observation, action, reward)
            break


turn: 0
old_location: 0
action: 2
new_location: 1
reward: 0.0
dead: False

turn: 1
old_location: 1
action: 0
new_location: 0
reward: 0.0
dead: False

turn: 2
old_location: 0
action: 2
new_location: 1
reward: 0.0
dead: False

turn: 3
old_location: 1
action: 1
new_location: 5
reward: 0.0
dead: False

turn: 4
old_location: 5
action: 1
new_location: 9
reward: 0.0
dead: False

turn: 5
old_location: 9
action: 1
new_location: 13
reward: 0.0
dead: False

turn: 6
old_location: 13
action: 0
new_location: 12
reward: 0.0
dead: False

turn: 7
old_location: 12
action: 2
new_location: 13
reward: 0.0
dead: False

turn: 8
old_location: 13
action: 0
new_location: 12
reward: 0.0
dead: False

turn: 9
old_location: 12
action: 0
new_location: 12
reward: 0.0
dead: False

turn: 10
old_location: 12
action: 1
new_location: 12
reward: 0.0
dead: False

turn: 11
old_location: 12
action: 1
new_location: 12
reward: 0.0
dead: False

turn: 12
old_location: 12
action: 0
new_location: 12
reward: 0.0
dead: False

turn: 1

## Evaluation
So it worked for this certain level. the weakness of my agent is when there are multiple moves that can kill you in on single state, it will only avoid the one it experienced first.

In [87]:
print(agent3.evaluate())

    Action  Reward
0      1.0      -1
1      2.0      -1
2      1.0      -1
3      0.0      -1
4      1.0      -1
5      0.0      -1
6      3.0      -1
7      NaN       0
8      1.0      -1
9      0.0      -1
10     1.0      -1
11     0.0      -1
12     3.0      -1
13     2.0      -1
14     NaN       0
15     1.0      -1
None


## Test Agent - Run 4
Let's see if it performs as well on a different level

Parameters
- `env.desc = (random)`
- `is_slippery = False`
- `exploration_rate = 0.0`
- Starting point = 0
- Goal = 15
- -1 reward for dying after an action

I expect the agent to get to the goal at some point. it might fail but as it plays it should learn where the holes are.

In [88]:
env = gym.make('FrozenLake-v1', 
               render_mode='human', 
               is_slippery = False)
env.reset()
env.render()

In [89]:
states = env.observation_space.n
agent4 = Agent_New(create_q_table(states), exploration_rate=0.0)

In [91]:
observation = 0  # We consider that we always start top left
max_moves = 100
episodes = 10
agent = agent4

for episode in range(episodes):
    env.reset()
    for move in range(max_moves):
        # Get an action based of the Q-table
        action = agent.step(observation)
        old_observation = observation

        # Execute action
        observation, reward, terminated, _, _ = env.step(action)

        # Print usefull info to track progress
        print(f'turn: {move+1}')
        print(f'old_location: {old_observation}')
        print(f'action: {action}')
        print(f'new_location: {observation}')
        print(f'reward: {reward}')
        print(f'dead: {terminated}')
        print()

        # stop if dead
        if terminated:
            reward = -1
            agent.learn(old_observation, action, reward)
            break

turn: 1
old_location: 0
action: 0
new_location: 0
reward: 0.0
dead: False

turn: 2
old_location: 0
action: 3
new_location: 0
reward: 0.0
dead: False

turn: 3
old_location: 0
action: 0
new_location: 0
reward: 0.0
dead: False

turn: 4
old_location: 0
action: 0
new_location: 0
reward: 0.0
dead: False

turn: 5
old_location: 0
action: 3
new_location: 0
reward: 0.0
dead: False

turn: 6
old_location: 0
action: 3
new_location: 0
reward: 0.0
dead: False

turn: 7
old_location: 0
action: 3
new_location: 0
reward: 0.0
dead: False

turn: 8
old_location: 0
action: 1
new_location: 4
reward: 0.0
dead: False

turn: 9
old_location: 4
action: 3
new_location: 0
reward: 0.0
dead: False

turn: 10
old_location: 0
action: 0
new_location: 0
reward: 0.0
dead: False

turn: 11
old_location: 0
action: 2
new_location: 1
reward: 0.0
dead: False

turn: 12
old_location: 1
action: 3
new_location: 1
reward: 0.0
dead: False

turn: 13
old_location: 1
action: 3
new_location: 1
reward: 0.0
dead: False

turn: 14
old_location

## Evaluation
Important note, we punish it by taking the gift for the first time, since it truncates just like it truncates on a death. so after getting it once it will never get it again.

In [92]:
agent3.evaluate()

    Action  Reward
0      NaN       0
1      1.0      -1
2      NaN       0
3      1.0      -1
4      2.0      -1
5      NaN       0
6      2.0      -1
7      NaN       0
8      1.0      -1
9      3.0      -1
10     2.0      -1
11     NaN       0
12     NaN       0
13     0.0      -1
14     2.0      -1
15     NaN       0


Succeeded on it's 11th episode on a new random level. I feel like this strategy is okayly reliable. would make it better if it kept track of everything that made it die instead of only 1. of course we could make it even better by using an actual q_table

## Test Agent - Run 5
I looked up some execution by someone else and got inspired

Source: https://www.youtube.com/watch?v=ZhoIgo3qqLU (quite recent video which is perfect)

The idea is to reward it for staying alive. First up i will try this by making everything a bad decision, if you stay alive you get positive feedback

### Create Environment

In [13]:
env = gym.make('FrozenLake-v1', render_mode='human', is_slippery = False)
env.reset()
env.render()

### Expand the q table

In [14]:
def create_bigger_q_table(states, actions):
    q = np.full((states, actions), -1) # 16 arrays
    return q

### Create Agent

In [15]:
class Agent_v3():  
    def __init__(self, q_table):
        self.q_table = q_table


    def step(self, state, exploration_rate):
        # Random action if exploring
        if rnd.randint(1, 100) <= (exploration_rate*100):
            print('Random action')
            return env.action_space.sample()
        
        # Find best action
        actions = self.q_table[state]
        best_action = env.action_space.sample()
        best_reward = [-1.0]
        for i, r in enumerate(actions):
            if r > best_reward:
                best_reward = r
                best_action = i
        print(f'best_reward: {best_reward}')
        print(f'best_action: {best_action}')
        return best_action



    def learn(self, state, action, reward):
        old_reward = self.q_table[state, action]
        if reward > old_reward:
            print("IM LEARNING")
            self.q_table[state, action] = reward



    def evaluate(self):
        df = pd.DataFrame(self.q_table)
        print(df)

        
agent5 = Agent_v3(create_bigger_q_table(env.observation_space.n, env.action_space.n))
agent5.q_table

array([[-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1],
       [-1, -1, -1, -1]])

In [16]:
observation = 0  # We consider that we always start top left
max_moves = 250
episodes = 1
exploration_rate=0.0

for episode in range(episodes):
    env.reset()
    for move in range(max_moves):
        
        # Uncomment for slow gameplay
        #time.sleep(1)
        
        # Get an action based of the Q-table
        action = agent5.step(observation, exploration_rate)
        old_observation = observation

        # Execute action
        observation, reward, terminated, _, _ = env.step(action)

        # Learn
        if (not terminated):
            agent5.learn(old_observation, action, reward)

        # Stop if dead
        if terminated:
            if reward == 0:
                reward = -1
            agent5.learn(old_observation, action, reward)
            break

best_reward: [-1.0]
best_action: 3
IM LEARNING
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
best_reward: 0
best_act

## Evaluation
Here we see the q table of the agent which can be read as the following:

```
        Actions ➝
States          
↓           rewards
```

you can see at Q(0, 3) that it gets a reward of 0 for moving up. this means it will always move up.

In [17]:
agent5.evaluate()

    0  1  2  3
0  -1 -1 -1  0
1  -1 -1 -1 -1
2  -1 -1 -1 -1
3  -1 -1 -1 -1
4  -1 -1 -1 -1
5  -1 -1 -1 -1
6  -1 -1 -1 -1
7  -1 -1 -1 -1
8  -1 -1 -1 -1
9  -1 -1 -1 -1
10 -1 -1 -1 -1
11 -1 -1 -1 -1
12 -1 -1 -1 -1
13 -1 -1 -1 -1
14 -1 -1 -1 -1
15 -1 -1 -1 -1


we can try to see how far we can get by tuning the exploration rate

In [106]:
observation = 0  # We consider that we always start top left
max_moves = 250
episodes = 10
exploration_rate=0.3

for episode in range(episodes):
    env.reset()
    for move in range(max_moves):
        
        # Uncomment for slow gameplay
        #time.sleep(1)
        
        # Get an action based of the Q-table
        action = agent5.step(observation, exploration_rate)
        old_observation = observation

        # Execute action
        observation, reward, terminated, _, _ = env.step(action)

        # Learn
        if (not terminated):
            agent5.learn(old_observation, action, reward)

        # Stop if dead
        if terminated:
            if reward == 0:
                reward = -1
            agent5.learn(old_observation, action, reward)
            break

best_reward: 0
best_action: 1
best_reward: 0
best_action: 0
best_reward: 0
best_action: 0
Random action
Random action
IM LEARNING
Random action
IM LEARNING
Random action
IM LEARNING
Random action
best_reward: 0
best_action: 3
best_reward: 0
best_action: 3
Random action
best_reward: [-1.0]
best_action: 3
IM LEARNING
best_reward: 0
best_action: 1
Random action
best_reward: 0
best_action: 1
best_reward: 0
best_action: 0
best_reward: 0
best_action: 0
best_reward: 0
best_action: 0
Random action
best_reward: 0
best_action: 0
best_reward: 0
best_action: 0
Random action
best_reward: 0
best_action: 0
Random action
best_reward: 0
best_action: 1
best_reward: 0
best_action: 0
best_reward: 0
best_action: 0
best_reward: 0
best_action: 0
best_reward: 0
best_action: 0
best_reward: 0
best_action: 0
best_reward: 0
best_action: 0
best_reward: 0
best_action: 0
best_reward: 0
best_action: 0
best_reward: 0
best_action: 0
best_reward: 0
best_action: 0
best_reward: 0
best_action: 0
best_reward: 0
best_action:

## Evaluation
The agent found more safe moves but its still staying quite close to its starting point

In [108]:
agent5.evaluate()

    0  1  2  3
0  -1  0  0  0
1   0 -1 -1  0
2  -1 -1 -1 -1
3  -1 -1 -1 -1
4   0  0 -1  0
5   0  0 -1  0
6  -1 -1 -1 -1
7  -1 -1 -1 -1
8  -1 -1 -1  0
9  -1 -1 -1 -1
10 -1 -1 -1 -1
11 -1 -1 -1 -1
12 -1  0 -1 -1
13 -1 -1 -1 -1
14 -1 -1 -1 -1
15 -1 -1 -1 -1


I have to come up with a different reward system

## Test Agent - Run 6 (Solid Agent)
I kept being stuck at the following dillema
- if we make the base q table value 0, we cant overwrite a -1 value (since -1 is not greater than 0) if we die so we prevent the action
- if we make the base q table value -1 it will find one good move and do it for the entire time

to fix this i will modify the agent a little

also very important is that we have to make the agent choose between the moves that have an equal value, currently it just chooses the same one over and over even if it has the same value as other actions. now im gonna make it that if 2 actions have the same value they will both have a chance to get chosen equally

#### Create Environment

In [122]:
env = gym.make('FrozenLake-v1', render_mode='human', is_slippery = False)
env.reset()
env.render()

#### Create Q-Table

In [123]:
def create_bigger_q_table(states, actions):
    q = np.full((states, actions), 0) # 16 arrays
    return q

#### Create Agent

In [129]:
class Agent_v4():  
    def __init__(self, q_table):
        self.q_table = q_table


    def step(self, state, exploration_rate):
        # Random action if exploring
        if rnd.randint(1, 100) <= (exploration_rate*100):
            print('Random action')
            return env.action_space.sample()
        
        # Find best action
        actions = self.q_table[state]
        best_action = env.action_space.sample()
        best_reward = [-1.0]
        good_actions = []
        for i, r in enumerate(actions):
            if r > best_reward:
                best_reward = r
            if r == best_reward:
                good_actions.append(i)
        print(f'good actions: {good_actions}')
        return rnd.choice(good_actions)



    def learn(self, state, action, reward):
        old_reward = self.q_table[state, action]
        if reward > old_reward:
            print("IM LEARNING")
            self.q_table[state, action] = reward


    def force_learn(self, state, action, reward):
        self.q_table[state, action] = reward



    def evaluate(self):
        df = pd.DataFrame(self.q_table)
        print(df)

        
agent6 = Agent_v4(create_bigger_q_table(env.observation_space.n, env.action_space.n))
agent6.q_table

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

THE run?

In [131]:
observation = 0  # We consider that we always start top left
max_moves = 100
episodes = 10
exploration_rate=0.0

for episode in range(episodes):
    env.reset()
    for move in range(max_moves):    
        # Get an action based of the Q-table
        action = agent6.step(observation, exploration_rate)
        old_observation = observation

        # Execute action
        observation, reward, terminated, _, _ = env.step(action)

        # Learn
        if (not terminated):
            agent6.learn(old_observation, action, reward)

        # Stop if dead
        if terminated:
            if reward == 0:
                reward = -1
            agent6.force_learn(old_observation, action, reward)
            break

good actions: [0, 1, 2, 3]
good actions: [0, 1, 3]
good actions: [0, 1, 3]
good actions: [0, 1, 3]
good actions: [0, 1, 3]
good actions: [0, 1, 3]
good actions: [0, 1, 3]
good actions: [0, 2, 3]
good actions: [0, 1, 3]
good actions: [0, 1, 3]
good actions: [0, 2, 3]
good actions: [0, 1, 2]
good actions: [0, 1, 3]
good actions: [0, 1, 3]
good actions: [0, 1, 2, 3]
good actions: [0, 2, 3]
good actions: [0, 2, 3]
good actions: [0, 1, 2, 3]
good actions: [0, 1, 2, 3]
good actions: [0, 2, 3]
good actions: [0, 1, 2, 3]
good actions: [0, 1, 2, 3]
good actions: [0, 1, 3]
good actions: [0, 2, 3]
good actions: [0, 2, 3]
good actions: [0, 2, 3]
good actions: [0, 1, 3]
good actions: [0, 1, 2, 3]
good actions: [0, 1, 2, 3]
good actions: [0, 1, 2, 3]
good actions: [0, 2, 3]
good actions: [0, 2, 3]
good actions: [0, 2, 3]
good actions: [0, 2, 3]
good actions: [0, 1, 2, 3]
good actions: [0, 1, 2, 3]
good actions: [0, 1, 2, 3]
good actions: [0, 1, 3]
good actions: [0, 1, 2, 3]
good actions: [0, 1, 2, 3

## Evaluation
And after some training... it works! the only problem is that it keeps walking into walls sometimes. we can fix it by making it HAVE to move. more about this in run7. 

in this evaluation you can exactly see where it will die (-1) and how to get the gift (1)

In [132]:
agent6.evaluate()

    0  1  2  3
0   0  0  0  0
1   0 -1  0  0
2   0  0  0  0
3   0 -1  0  0
4   0  0 -1  0
5   0  0  0  0
6  -1  0 -1  0
7   0  0  0  0
8   0 -1  0  0
9   0  0  0 -1
10  0  0 -1  0
11  0  0  0  0
12  0  0  0  0
13 -1  0  0  0
14  0  0  1  0
15  0  0  0  0


## Test Agent - Run 7 (Pretty Solid)
For optimalisation we will punish the agent for staying on the same tile.

#### Create Environment

In [193]:
env = gym.make('FrozenLake-v1', is_slippery = False)
env.reset()
env.render()

c:\Users\kaan-\anaconda3\envs\gymenv38\lib\site-packages\gym\envs\toy_text\frozen_lake.py:271: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("FrozenLake-v1", render_mode="rgb_array")
  logger.warn(


#### Create Q-Table

In [194]:
def create_bigger_q_table(states, actions):
    q = np.full((states, actions), 0) # 16 arrays
    return q

#### Create Agent

In [195]:
class Agent_v5():  
    def __init__(self, q_table):
        self.q_table = q_table


    def step(self, state, exploration_rate):
        # Random action if exploring
        if rnd.randint(1, 100) <= (exploration_rate*100):
            print('Random action')
            return env.action_space.sample()
        
        # Find best action
        actions = self.q_table[state]
        best_reward = [-1.0]
        good_actions = []
        for i, r in enumerate(actions):
            if r > best_reward:
                print(r)
                print('reset')
                best_reward = r
                good_actions.clear()
            if r == best_reward:
                ('add it')
                good_actions.append(i)
        print(f'good actions: {good_actions}')
        return rnd.choice(good_actions)



    def learn(self, state, action, reward):
        old_reward = self.q_table[state, action]
        if reward > old_reward:
            print("IM LEARNING")
            self.q_table[state, action] = reward


    def force_learn(self, state, action, reward):
        print(reward)
        self.q_table[state, action] = reward



    def evaluate(self):
        df = pd.DataFrame(self.q_table)
        print(df)

        
agent7 = Agent_v5(create_bigger_q_table(env.observation_space.n, env.action_space.n))
agent7.q_table

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [200]:
observation = 0  # We consider that we always start top left
max_moves = 100
episodes = 100
exploration_rate=0.0
agent = agent7

for episode in range(episodes):
    env.reset()
    for move in range(max_moves):    

        # Get an action based of the Q-table
        action = agent.step(observation, exploration_rate)
        old_observation = observation

        # Execute action
        observation, reward, terminated, _, _ = env.step(action)

        # Learn
        if (not terminated):
            if observation == old_observation:
                print('same')
                reward = -1
                agent.force_learn(old_observation, action, reward)
            else:
                agent.learn(old_observation, action, reward)

        # Stop if dead
        if terminated:
            if reward == 0:
                reward = -2
            agent.force_learn(old_observation, action, reward)
            break

0
reset
good actions: [1, 2]
0
reset
good actions: [0, 2]
0
reset
good actions: [0, 1, 2]
0
reset
good actions: [1, 3]
0
reset
good actions: [0, 1, 2]
0
reset
good actions: [0]
0
reset
good actions: [0, 1, 2]
0
reset
good actions: [0, 2]
0
reset
good actions: [0, 1, 2]
0
reset
good actions: [0, 2]
0
reset
good actions: [0, 1, 2]
0
reset
good actions: [0]
0
reset
good actions: [0, 1, 2]
0
reset
good actions: [1, 3]
0
reset
good actions: [0, 1, 3]
0
reset
1
reset
good actions: [2]
1.0
0
reset
good actions: [0, 1, 2, 3]
0
reset
good actions: [1, 2]
0
reset
good actions: [0, 2]
0
reset
good actions: [0, 1, 2]
0
reset
good actions: [1, 3]
0
reset
good actions: [0, 1, 2]
0
reset
good actions: [0]
0
reset
good actions: [0, 1, 2]
0
reset
good actions: [0]
0
reset
good actions: [0, 1, 2]
0
reset
good actions: [0, 2]
0
reset
good actions: [0, 1, 2]
0
reset
good actions: [0]
0
reset
good actions: [0, 1, 2]
0
reset
good actions: [0, 2]
0
reset
good actions: [1, 2]
0
reset
good actions: [0, 2]
0
re

## Evaluation
It worked, it now doesnt walk into the walls all the time. in the table below you can see
- -2: dying
- -1: walking into a wall
- +0: walking 
- +1: getting the gift

In [201]:
agent7.evaluate()

    0  1  2  3
0  -1  0  0 -1
1   0 -2  0 -1
2   0  0  0 -1
3   0 -2 -1 -1
4  -1  0 -2  0
5   0  0  0  0
6  -2  0 -2  0
7   0  0  0  0
8  -1 -2  0  0
9   0  0  0 -2
10  0  0 -2  0
11  0  0  0  0
12  0  0  0  0
13 -2 -1  0  0
14  0  0  1  0
15  0  0  0  0


## Test Agent - Run 8 (Amazing)

For optimalisation we will punish the agent for staying on the same tile.
I have one final idea to optimise it even more, we can try to make the agent calculate steps ahead. for example, 

currently the agent understands that if its on file 14, it can obtain the gift by moving right. if it can look into the future, it can understand it is a good idea to move onto file 14, bceause file 14 will result into a gift.

To do this I will increase the reward of the gift to 3. every step back in time will reduce the reward by 1, so that it doesnt decide to walk back if it is on its way to the gift.

#### Create Environment

In [205]:
env = gym.make('FrozenLake-v1', is_slippery = False)
env.reset()
env.render()

c:\Users\kaan-\anaconda3\envs\gymenv38\lib\site-packages\gym\envs\toy_text\frozen_lake.py:271: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("FrozenLake-v1", render_mode="rgb_array")
  logger.warn(


#### Create Q-Table

In [4]:
def create_bigger_q_table(states, actions):
    q = np.full((states, actions), 0) # 16 arrays
    return q

#### Create Agent

In [5]:
class Agent_v6():  
    def __init__(self, q_table):
        self.q_table = q_table


    def step(self, state, exploration_rate):
        # Random action if exploring
        if rnd.randint(1, 100) <= (exploration_rate*100):
            print('Random action')
            return env.action_space.sample()
        
        # Find best action
        actions = self.q_table[state]
        best_reward = [-1.0]
        good_actions = []
        for i, r in enumerate(actions):
            if r > best_reward:
                best_reward = r
                good_actions.clear()
            if r == best_reward:
                ('add it')
                good_actions.append(i)
        print(f'good actions: {good_actions}')
        choice = rnd.choice(good_actions)
        print(f'chosen action {choice}')
        return choice



    def learn(self, state, action, reward):
        old_reward = self.q_table[state, action]
        if reward > old_reward:
            print("IM LEARNING")
            self.q_table[state, action] = reward


    def force_learn(self, state, action, reward):
        print(reward)
        self.q_table[state, action] = reward



    def evaluate(self):
        df = pd.DataFrame(self.q_table)
        print(df)

agent8 = Agent_v6(create_bigger_q_table(env.observation_space.n, env.action_space.n))
agent8.q_table

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0,

Train/Test (looking 2 steps into the future) (hardcoded)

In [239]:
observation = 0  # We consider that we always start top left
max_moves = 100
episodes = 100
exploration_rate=0.0
agent = agent8

for episode in range(episodes):
    env.reset()
    for move in range(max_moves):  

        time.sleep(0)  

        # Get an action based of the Q-table
        if (move > 1):
            older_action = old_action
        if (move > 0):
            old_action = action
        action = agent.step(observation, exploration_rate)

        if (move > 1):
            oldest_observation = older_observation
        if (move > 0):
            older_observation = old_observation
        old_observation = observation

        # Execute action
        observation, reward, terminated, _, _ = env.step(action)

        # Learn
        if (not terminated):
            if observation == old_observation:
                reward = -1
                agent.force_learn(old_observation, action, reward)
            else:
                agent.learn(old_observation, action, reward)

        # Stop if dead
        if terminated:
            if reward == 0:
                reward = -2
            else:
                reward = 3
                agent.force_learn(older_observation, old_action, reward-1)
                agent.force_learn(oldest_observation, older_action, reward-2)
            agent.force_learn(old_observation, action, reward)
            break

good actions: [1, 2]
chosen action 1
good actions: [3]
chosen action 3
good actions: [1, 2]
chosen action 2
good actions: [0, 2]
chosen action 2
good actions: [0, 1, 2]
chosen action 2
good actions: [0]
chosen action 0
good actions: [0, 1, 2]
chosen action 1
good actions: [1, 3]
chosen action 1
good actions: [1]
chosen action 1
good actions: [2]
chosen action 2
2
1
3
good actions: [0, 1, 2, 3]
chosen action 1
good actions: [3]
chosen action 3
good actions: [1, 2]
chosen action 1
good actions: [3]
chosen action 3
good actions: [1, 2]
chosen action 2
good actions: [0, 2]
chosen action 2
good actions: [0, 1, 2]
chosen action 2
good actions: [0]
chosen action 0
good actions: [0, 1, 2]
chosen action 2
good actions: [0]
chosen action 0
good actions: [0, 1, 2]
chosen action 2
good actions: [0]
chosen action 0
good actions: [0, 1, 2]
chosen action 1
good actions: [1]
chosen action 1
good actions: [1]
chosen action 1
good actions: [2]
chosen action 2
2
1
3
good actions: [0, 1, 2, 3]
chosen acti

## Evaluation
Perfect
- -2: will result in death
- -1: will result in standing still
- 0: just walk
- 1: will result in being 2 steps away from gift
- 2: will result in being 1 step away from gift
- 3: will result in getting gift

In [240]:
agent8.evaluate()

    0  1  2  3
0  -1  0  0 -1
1   0 -2  0 -1
2   0  0  0 -1
3   0 -2 -1 -1
4  -1 -1 -2  0
5   0  0  0  0
6  -2  1 -2  0
7   0  0  0  0
8  -1 -2  0  0
9   0  0  0 -2
10  0  2 -2  0
11  0  0  0  0
12  0  0  0  0
13 -2  0  0  0
14  0  0  3  0
15  0  0  0  0


## Test Agent - Run 9 (Amazing)

quickly coded it to look 3 steps into the future. But of course it would be ideal to make it a hyper parameter

In [249]:
agent9 = Agent_v6(create_bigger_q_table(env.observation_space.n, env.action_space.n))
agent9.q_table

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [254]:
env = gym.make('FrozenLake-v1', is_slippery = False)
env.reset()
env.render()

new_observation = 0  # We consider that we always start top left
max_moves = 100
episodes = 100
exploration_rate=0.0
agent = agent9
steps_looking_into_future = 2

for episode in range(episodes):
    env.reset()
    for move in range(max_moves):  

        time.sleep(0)  

        # remember actions
        if (move > 2):
            action_min_3 = action_min_2
        if (move > 1):
            action_min_2 = action_min_1
        if (move > 0):
            action_min_1 = action
        # Get an action based of the Q-table
        action = agent.step(new_observation, exploration_rate)

        # remember moves
        if (move > 2):
            observation_min_3 = observation_min_2
        if (move > 1):
            observation_min_2 = observation_min_1
        if (move > 0):
            observation_min_1 = observation
        observation = new_observation

        # Execute action
        new_observation, reward, terminated, _, _ = env.step(action)

        # Learn
        if (not terminated):
            if new_observation == observation:
                reward = -1
                agent.force_learn(observation, action, reward)
            else:
                agent.learn(observation, action, reward)

        # Stop if dead
        if terminated:
            if reward == 0:
                reward = -2
            else:
                reward = 4
                agent.force_learn(observation_min_1, action_min_1, reward-1)
                agent.force_learn(observation_min_2, action_min_2, reward-2)
                agent.force_learn(observation_min_3, action_min_3, reward-3)
            agent.force_learn(observation, action, reward)
            break

good actions: [1, 2]
chosen action 2
good actions: [0, 2]
chosen action 2
good actions: [1]
chosen action 1
good actions: [1]
chosen action 1
good actions: [1]
chosen action 1
good actions: [2]
chosen action 2
3
2
1
4
good actions: [0, 1, 2, 3]
chosen action 0
good actions: [1, 2]
chosen action 1
good actions: [1, 3]
chosen action 1
good actions: [2]
chosen action 2
good actions: [2]
chosen action 2
good actions: [1]
chosen action 1
good actions: [2]
chosen action 2
3
2
1
4
good actions: [0, 1, 2, 3]
chosen action 3
good actions: [1, 2]
chosen action 1
good actions: [1, 3]
chosen action 3
good actions: [1, 2]
chosen action 2
good actions: [0, 2]
chosen action 0
good actions: [1, 2]
chosen action 2
good actions: [0, 2]
chosen action 2
good actions: [1]
chosen action 1
good actions: [1]
chosen action 1
good actions: [1]
chosen action 1
good actions: [2]
chosen action 2
3
2
1
4
good actions: [0, 1, 2, 3]
chosen action 3
good actions: [1, 2]
chosen action 2
good actions: [0, 2]
chosen acti

c:\Users\kaan-\anaconda3\envs\gymenv38\lib\site-packages\gym\envs\toy_text\frozen_lake.py:271: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("FrozenLake-v1", render_mode="rgb_array")
  logger.warn(


## Evaluation
- -2: will result in death
- -1: will result in standing still
- 0: just walk
- 1: will result in being 3 steps away from gift
- 2: will result in being 2 step away from gift
- 2: will result in being 1 step away from gift
- 4: will result in getting gift

In [255]:
agent9.evaluate()

    0  1  2  3
0  -1  0  0 -1
1   0 -2  0 -1
2   0  1  0 -1
3   0 -2 -1 -1
4  -1  0 -2  0
5   0  0  0  0
6  -2  2 -2  0
7   0  0  0  0
8   0 -2  1  0
9   0  0  2 -2
10  0  3 -2  0
11  0  0  0  0
12  0  0  0  0
13  0  0  3  0
14  2 -1  4  0
15  0  0  0  0


## Test Agent - Run 10 (Amazing)

In [320]:
agent10 = Agent_v6(create_bigger_q_table(env.observation_space.n, env.action_space.n))
agent10.q_table

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

**my head is borderline exploding trying to make this a hyper parameter (its still hardcoded)**

here is hardcoded 5 steps

In [325]:
env = gym.make('FrozenLake-v1', render_mode='human', is_slippery = False)
#env = gym.make('FrozenLake-v1', is_slippery = False)
env.reset()
env.render()

new_observation = 0  # We consider that we always start top left
max_moves = 100
episodes = 3
exploration_rate=0.0
agent = agent10
steps_looking_into_future = 5
action_min_3 = None

for episode in range(episodes):
    env.reset()
    for move in range(max_moves):  

        # remember moves
        if (move > 4):
            action_min_5 = action_min_4
        if (move > 3):
            action_min_4 = action_min_3
        if (move > 2):
            action_min_3 = action_min_2
        if (move > 1):
            action_min_2 = action_min_1
        if (move > 0):
            action_min_1 = action

        # Get an action based of the Q-table
        action = agent.step(new_observation, exploration_rate)

        # remember moves
        if (move > 4):
            observation_min_5 = observation_min_4
        if (move > 3):
            observation_min_4 = observation_min_3
        if (move > 2):
            observation_min_3 = observation_min_2
        if (move > 1):
            observation_min_2 = observation_min_1
        if (move > 0):
            observation_min_1 = observation
        observation = new_observation

        # Execute action
        new_observation, reward, terminated, _, _ = env.step(action)

        # Learn
        if (not terminated):
            if new_observation == observation:
                reward = -1
                agent.force_learn(observation, action, reward)
            else:
                agent.learn(observation, action, reward)

        # Stop if dead
        if terminated:
            if reward == 0:
                reward = -2
            else:
                reward = 6
                agent.force_learn(observation_min_1, action_min_1, reward-1)
                agent.force_learn(observation_min_2, action_min_2, reward-2)
                agent.force_learn(observation_min_3, action_min_3, reward-3)
                agent.force_learn(observation_min_4, action_min_4, reward-4)
                agent.force_learn(observation_min_5, action_min_5, reward-5)
            agent.force_learn(observation, action, reward)
            break

good actions: [1]
chosen action 1
good actions: [1]
chosen action 1
good actions: [2]
chosen action 2
good actions: [2]
chosen action 2
good actions: [1]
chosen action 1
good actions: [2]
chosen action 2
5
4
3
2
1
6
good actions: [1]
chosen action 1
good actions: [1]
chosen action 1
good actions: [2]
chosen action 2
good actions: [2]
chosen action 2
good actions: [1]
chosen action 1
good actions: [2]
chosen action 2
5
4
3
2
1
6
good actions: [1]
chosen action 1
good actions: [1]
chosen action 1
good actions: [2]
chosen action 2
good actions: [2]
chosen action 2
good actions: [1]
chosen action 1
good actions: [2]
chosen action 2
5
4
3
2
1
6


## Evaluation
- -2: will result in death
- -1: will result in standing still
- 0: just walk
- 1: will result in being 5 steps away from gift
- 2: will result in being 4 step away from gift
- 3: will result in being 3 step away from gift
- 4: will result in being 2 step away from gift
- 5: will result in being 1 step away from gift
- 6: will result in getting gift

In [323]:
agent10.evaluate()

    0  1  2  3
0  -1  1  0 -1
1   0 -2  0 -1
2   0  0  0 -1
3   0 -2 -1 -1
4  -1  2 -2  0
5   0  0  0  0
6  -2  4 -2  0
7   0  0  0  0
8  -1 -2  3  0
9   0  0  4 -2
10  0  5 -2  3
11  0  0  0  0
12  0  0  0  0
13 -2  0  0  0
14  0 -1  6  0
15  0  1  0  0


## Test Agent - Run 11 (Perfect)
trying to remove the hardcoded looking-steps-into-the-future and make it a hyper parameter

In [326]:
agent11 = Agent_v6(create_bigger_q_table(env.observation_space.n, env.action_space.n))
agent11.q_table

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [333]:
env = gym.make('FrozenLake-v1', render_mode='human', is_slippery = False)
#env = gym.make('FrozenLake-v1', is_slippery = False)
env.reset()
env.render()

new_observation = 0  # We consider that we always start top left
max_moves = 100 # max moves before resetting env
episodes = 1 # amouint of tries
exploration_rate=0.0 # the chance of ignoring what we've learned and doing a randoma action
steps_looking_into_future = 5 # the amount of steps we should look into the future
action_history = []
state_history = []
agent = agent11

for episode in range(episodes):
    env.reset()
    for move in range(max_moves):  

        if move>0:
            if len(action_history) < steps_looking_into_future:
                action_history.append(action)
            else: 
                action_history.pop(0)
                action_history.append(action)

        # Get an action based of the Q-table
        action = agent.step(new_observation, exploration_rate)

        if move>0:
            if len(state_history) < steps_looking_into_future:
                state_history.append(observation)
            else:
                state_history.pop(0)
                state_history.append(observation)

        observation = new_observation

        # Execute action
        new_observation, reward, terminated, _, _ = env.step(action)

        # Learn
        if (not terminated):
            if new_observation == observation:
                reward = -1
                agent.force_learn(observation, action, reward)
            else:
                agent.learn(observation, action, reward)

        # Stop if dead
        if terminated:
            if reward == 0:
                reward = -2
            else:
                reward = steps_looking_into_future+1
                for x, _ in enumerate(range(steps_looking_into_future)):
                    agent.force_learn(state_history[x], action_history[x], reward - (x+1))
            agent.force_learn(observation, action, reward)
            break

good actions: [1]
chosen action 1
good actions: [1]
chosen action 1
good actions: [2]
chosen action 2
good actions: [1]
chosen action 1
good actions: [2]
chosen action 2
good actions: [2]
chosen action 2
5
4
3
2
1
6


## Evaluation
- -2: will result in death
- -1: will result in standing still
- 0: just walk
- 1: will result in being 5 steps away from gift
- 2: will result in being 4 step away from gift
- 3: will result in being 3 step away from gift
- 4: will result in being 2 step away from gift
- 5: will result in being 1 step away from gift
- 6: will result in getting gift

In [330]:
agent11.evaluate()

    0  1  2  3
0  -1  5  0 -1
1   0 -2  0 -1
2   0  0  0 -1
3   0 -2 -1 -1
4  -1  4 -2  0
5   0  0  0  0
6  -2  5  0  0
7   0  0  0  0
8  -1 -2  3  0
9   0  2  0 -2
10  0  4 -2  0
11  0  0  0  0
12  0  0  0  0
13 -2 -1  1  0
14  0  0  6  0
15  0  5  0  0


## Test Agent - Run 12 (THE ULTIMATE TEST)
Here is the ultimate test. Will my agent be able to compltely nail a 8x8 grid? let's find out

In [13]:
#env = gym.make('FrozenLake-v1', render_mode='human', is_slippery = False, map_name='8x8')
env = gym.make('FrozenLake-v1', is_slippery = False, map_name='8x8', desc=[
    "SFFFFFFF",
    "FHFHFHFH",
    "FHFHFFFF",
    "FHFHHHFH",
    "FFFHFFFF",
    "FHHFFHHF",
    "FHFFFFHH",
    "FFFHFFFG",])
env.reset()
env.render()

print(env.observation_space.n)
print(env.action_space.n)

64
4


c:\Users\kaan-\anaconda3\envs\gymenv38\lib\site-packages\gym\envs\toy_text\frozen_lake.py:271: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("FrozenLake-v1", render_mode="rgb_array")
  logger.warn(


In [44]:
agent11 = Agent_v6(create_bigger_q_table(env.observation_space.n, env.action_space.n))

on a 4x4 grid it was enough to look 5 steps into the future for an always perfect score. now the grid is bigger, which means we have to look more steps into the future. let's try 12

In [29]:
env = gym.make('FrozenLake-v1', render_mode='human', is_slippery = False, map_name='8x8', 
               desc=["SFFFFFFF",
               "FHFHFHFH",
               "FHFHFFFF",
               "FHFHHHFH",
               "FFFHFFFF",
               "FHHFFHHF",
               "FHFFFFHH",
               "FFFHFFFG",])

In [24]:
env = gym.make('FrozenLake-v1', is_slippery = False, map_name='8x8', 
               desc=["SFFFFFFF",
               "FHFHFHFH",
               "FHFHFFFF",
               "FHFHHHFH",
               "FFFHFFFF",
               "FHHFFHHF",
               "FHFFFFHH",
               "FFFHFFFG",])

In [50]:
env = gym.make('FrozenLake-v1', render_mode='human', is_slippery = False, map_name='8x8',
               desc=["SFFHHFHF",
"HFHHFFHF",
"FFHFFHHH",
"HFFFFHHF",
"HFFFHFHH",
"HFHFFHFH",
"FFHHFFFH",
"HFHFHHFG"]) 

In [46]:
env = gym.make('FrozenLake-v1', is_slippery = False, map_name='8x8',                desc=["SFFHHFHF",
"HFHHFFHF",
"FFHFFHHH",
"HFFFFHHF",
"HFFFHFHH",
"HFHFFHFH",
"FFHHFFFH",
"HFHFHHFG"]) 

In [51]:
env.reset()
env.render()

max_moves = 1000 # max moves before resetting env
episodes = 3 # amouint of tries
exploration_rate=0.0 # the chance of ignoring what we've learned and doing a randoma action
steps_looking_into_future = 13 # the amount of steps we should look into the future
agent = agent11

for episode in range(episodes):
    new_observation = 0  # We consider that we always start top left
    action_history = []
    state_history = []  
    env.reset()
    for move in range(max_moves):  
        print(f'episode: {episode}')
        print(f'move: {move}')
        if move>0:
            if len(action_history) < steps_looking_into_future:
                action_history.append(action)
            else: 
                action_history.pop(0)
                action_history.append(action)

        # Get an action based of the Q-table
        print(f'state: {new_observation}')
        action = agent.step(new_observation, exploration_rate)

        if move>0:
            if len(state_history) < steps_looking_into_future:
                state_history.append(observation)
            else:
                state_history.pop(0)
                state_history.append(observation)

        observation = new_observation

        # Execute action
        new_observation, reward, terminated, truncated, _ = env.step(action)

        # Learn
        if (not terminated):
            if new_observation == observation:
                reward = -1
                agent.force_learn(observation, action, reward)
            else:
                agent.learn(observation, action, reward)

        print(f'reward by game: {reward}')

        # Stop if dead
        if terminated:
            if reward == 0 and move != max_moves:
                reward = -2
            elif reward == 1:
                print('POG CHAMP MOMENT')
                reward = steps_looking_into_future+1
                for x, _ in enumerate(range(steps_looking_into_future)):
                    new_reward = reward - (reward-x-1)
                    print(f'reward: {new_reward}')
                    # agent.force_learn(state_history[x], action_history[x], new_reward)
                    agent.learn(state_history[x], action_history[x], new_reward)
            else:
                reward = 0
            print(f'last: {observation}, {action}')
            agent.force_learn(observation, action, reward)
            break

episode: 0
move: 0
state: 0
good actions: [2]
chosen action 2
reward by game: 0.0
episode: 0
move: 1
state: 1
good actions: [1]
chosen action 1
reward by game: 0.0
episode: 0
move: 2
state: 9
good actions: [1]
chosen action 1
reward by game: 0.0
episode: 0
move: 3
state: 17
good actions: [1]
chosen action 1
reward by game: 0.0
episode: 0
move: 4
state: 25
good actions: [2]
chosen action 2
reward by game: 0.0
episode: 0
move: 5
state: 26
good actions: [2]
chosen action 2
reward by game: 0.0
episode: 0
move: 6
state: 27
good actions: [1]
chosen action 1
reward by game: 0.0
episode: 0
move: 7
state: 35
good actions: [1]
chosen action 1
reward by game: 0.0
episode: 0
move: 8
state: 43
good actions: [2]
chosen action 2
reward by game: 0.0
episode: 0
move: 9
state: 44
good actions: [1]
chosen action 1
reward by game: 0.0
episode: 0
move: 10
state: 52
good actions: [2]
chosen action 2
reward by game: 0.0
episode: 0
move: 11
state: 53
good actions: [2]
chosen action 2
reward by game: 0.0
episo

# AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA IT WORKS HAHAHAHAHAHAHAHAHHHHHHHAHHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHAHHAHAHAHAHAHAHA LESGOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO

## Evaluation
- -2: will result in death
- -1: will result in standing still
- 0: just walk
- 1: will result in being 15 steps away from gift
- 2: will result in being 14 step away from gift
- 3: will result in being 13 steps away from gift
- 4: will result in being 12 steps away from gift
- 5: will result in being 11 step away from gift
- 6: will result in being 10 steps away from gift
- 7: will result in being 9 step away from gift
- 8: will result in being 8 step away from gift
- 9: will result in being 7 step away from gift
- 10: will result in being 6 step away from gift
- 11: will result in being 5 steps away from gift
- 12: will result in being 4 step away from gift
- 13: will result in being 3 step away from gift
- 14: will result in being 2 step away from gift
- 15: will result in being 1 step away from gift
- 16: will result in getting gift

In [52]:
pd.set_option('display.max_rows', None)
print((agent11.evaluate()))

    0   1   2  3
0  -1  -2   1 -1
1   0   2   0 -1
2   0  -2  -2 -1
3   0   0   0  0
4   0   0   0  0
5  -2   0  -2 -1
6   0   0   0  0
7   0   0   0  0
8   0   0   0  0
9  -2   3  -2  0
10  0   0   0  0
11  0   0   0  0
12 -2   0   0 -2
13  0  -2  -2  0
14  0   0   0  0
15  0   0   0  0
16 -1  -2   3 -2
17  2   6  -2  0
18  0   0   0  0
19 -2   0   0 -2
20  0   0  -2  0
21  0   0   0  0
22  0   0   0  0
23  0   0   0  0
24  0   0   0  0
25 -2   0   7  5
26  0   0   8 -2
27  0   9   0  0
28  0  -2  -2  0
29  0   0   0  0
30  0   0   0  0
31  0   0   0  0
32  0   0   0  0
33 -2   0   0  0
34  0  -2   0  3
35  2  10  -2  0
36  0   0   0  0
37  0   0   0  0
38  0   0   0  0
39  0   0   0  0
40  0   0   0  0
41 -2   0  -2  0
42  0   0   0  0
43 -2  -2  11  9
44  8  12  -2 -2
45  0   0   0  0
46 -2   0   0  0
47  0   0   0  0
48 -1  -2   0 -2
49  0   0  -2  0
50  0   0   0  0
51  0   0   0  0
52 -2  -2  13  0
53  0  -2  14 -2
54  0  15  -2  0
55  0   0   0  0
56  0   0   0  0
57 -2  -1  -2 

nah i swear you dont know how proud i am rn